# Glance
A graphical anomaly detection engine for real world sensor data.

This is a new approach to a common problem in manufacturing and industrial control. Remote sensors provide monitoring o fprocess parameters but it can be a challenge to identify anomalies which occur rarely and can be subtle in real world noisy data. By approaching this classification as an image classification with CNNs we can leverage some of the powerful intuition that human visual 

In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline